In [8]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import pandas as pd
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import f1_score, jaccard_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading data and splitting into train, test and validation

In [10]:
H = 256
W = 256

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_dataset(path, split=0.2):
    images = sorted(glob(os.path.join(path, "images", "*.jpg")))
    masks = sorted(glob(os.path.join(path, "liver_masks", "*.jpg")))

    split_size = int(len(images) * split)

    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=split_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=split_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)


def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (h, w)
    x = cv2.resize(x, (W, H))   ## (h, w)
    x = x / 255.0               ## (h, w)
    x = x.astype(np.float32)    ## (h, w)
    x = np.expand_dims(x, axis=-1)## (h, w, 1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset

np.random.seed(42)
tf.random.set_seed(42)
""" Directory for storing files """
create_dir("files")

""" Hyperparameters """
batch_size = 2
lr = 1e-4
num_epochs = 50
model_path = os.path.join("files", "model.h5")
csv_path = os.path.join("files", "log.csv")

""" Dataset """
# dataset_path = "F:/JU PROJECTS/LiverDatasets/3DIRCADB manual"
dataset_path = "/content/drive/MyDrive/3DIRCADB manual"
# dataset_path = "/kaggle/input/3dircadb"
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")
print(f"Test : {len(test_x)} - {len(test_y)}")

train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

Train: 1695 - 1695
Valid: 564 - 564
Test : 564 - 564


Metrics

In [11]:
train_y[10]

'/content/drive/MyDrive/3DIRCADB manual/liver_masks/301.jpg'

In [12]:
smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

Model

In [13]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, 2, strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)


    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model

model = build_unet((H, W, 3))
model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef])
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_19 (Conv2D)             (None, 256, 256, 64  1792        ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_18 (BatchN  (None, 256, 256, 64  256        ['conv2d_19[0][0]']              
 ormalization)                  )                                                              

In [14]:
callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
    CSVLogger(csv_path),
    EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=False),
]

model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    callbacks=callbacks
)

Epoch 1/50
848/848 [==============================] - ETA: 0s - loss: 0.6142 - dice_coef: 0.3861
Epoch 1: val_loss improved from inf to 0.51068, saving model to files/model.h5
848/848 [==============================] - 920s 1s/step - loss: 0.6142 - dice_coef: 0.3861 - val_loss: 0.5107 - val_dice_coef: 0.4893 - lr: 1.0000e-04
Epoch 2/50
848/848 [==============================] - ETA: 0s - loss: 0.4553 - dice_coef: 0.5450
Epoch 2: val_loss improved from 0.51068 to 0.39991, saving model to files/model.h5
848/848 [==============================] - 143s 168ms/step - loss: 0.4553 - dice_coef: 0.5450 - val_loss: 0.3999 - val_dice_coef: 0.6001 - lr: 1.0000e-04
Epoch 3/50
848/848 [==============================] - ETA: 0s - loss: 0.3839 - dice_coef: 0.6163
Epoch 3: val_loss improved from 0.39991 to 0.33764, saving model to files/model.h5
848/848 [==============================] - 135s 159ms/step - loss: 0.3839 - dice_coef: 0.6163 - val_loss: 0.3376 - val_dice_coef: 0.6624 - lr: 1.0000e-04
Epoch

Predict

In [16]:
def save_results(image, mask, y_pred, save_image_path):
    mask = np.expand_dims(mask, axis=-1)
    mask = np.concatenate([mask, mask, mask], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)
    y_pred = y_pred * 255

    line = np.ones((H, 10, 3)) * 255
    
                    
    cat_images = np.concatenate([image, line, mask, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)
  
""" Directory for storing files """
create_dir("results")

""" Load the model """
with CustomObjectScope({"dice_coef": dice_coef, "dice_loss": dice_loss}):
    model = tf.keras.models.load_model(os.path.join("files", "model.h5"))

""" Prediction and Evaluation """
SCORE = []
for x, y in tqdm(zip(test_x, test_y), total=len(test_y)):
    """ Extracting the name """
    name = x.split("/")[-1]

    """ Reading the image """
    image = cv2.imread(x, cv2.IMREAD_COLOR) ## [W, H, 3]
    image = cv2.resize(image, (W, H))       ## [W, H, 3]
    x = image/255.0                         ## [W, H, 3]
    x = np.expand_dims(x, axis=0)           ## [1, W, H, 3]

    """ Reading the mask """
    mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (W, H))

    """ Prediction """
    y_pred = model.predict(x, verbose=0)[0]
    y_pred = np.squeeze(y_pred, axis=-1)
    y_pred = y_pred >= 0.5
    y_pred = y_pred.astype(np.int32)

    """ Saving the prediction """
    save_image_path = os.path.join("results", name)
    save_results(image, mask, y_pred, save_image_path)

    """ Flatten the array """
    mask = mask/255.0
    mask = (mask > 0.5).astype(np.int32).flatten()
    y_pred = y_pred.flatten()

    """ Calculating the metrics values """
    f1_value = f1_score(mask, y_pred, labels=[0, 1], average="binary")
    jac_value = jaccard_score(mask, y_pred, labels=[0, 1], average="binary")
    recall_value = recall_score(mask, y_pred, labels=[0, 1], average="binary", zero_division=0)
    precision_value = precision_score(mask, y_pred, labels=[0, 1], average="binary", zero_division=0)
    SCORE.append([name, f1_value, jac_value, recall_value, precision_value])

  0%|          | 1/564 [00:01<09:46,  1.04s/it]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  0%|          | 2/564 [00:01<07:28,  1.25it/s]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score

In [17]:
""" Metrics values """
score = [s[1:]for s in SCORE]
score = np.mean(score, axis=0)
print(f"F1: {score[0]:0.5f}")
print(f"Jaccard: {score[1]:0.5f}")
print(f"Recall: {score[2]:0.5f}")
print(f"Precision: {score[3]:0.5f}")

df = pd.DataFrame(SCORE, columns=["Image", "F1", "Jaccard", "Recall", "Precision"])
df.to_csv("files/score.csv")

F1: 0.52057
Jaccard: 0.49516
Recall: 0.52423
Precision: 0.52447


In [31]:
create_dir("results2")

In [32]:
def save_results(image, output, save_image_path):
    
    output = np.expand_dims(output, axis=-1)
    output = np.concatenate([output, output, output], axis=-1)
    output = output * 255

    for i in range(0, 256):
        for j in range(0, 256):
            for k in range(0, 3):
                if output[i][j][k] == 255:
                    output[i][j][k] = image[i][j][k] 
    line = np.ones((H, 10, 3)) * 255
                    
    cat_images = np.concatenate([image, line, output], axis=1)
    cv2.imwrite(save_image_path, cat_images)
    
  
create_dir("results2")

""" Directory for storing files """   
image_directory = "/content/drive/MyDrive/3DIRCADB manual/images/"
dataset = []
images = os.listdir(image_directory)
for i, img_name in tqdm(enumerate(images)):
    image = cv2.imread(image_directory + img_name, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (W, H))       ## [W, H, 3]
    x = image/255.0                         ## [W, H, 3]
    x = np.expand_dims(x, axis=0) 
    
    """ Prediction """
    y_pred = model.predict(x, verbose=0)[0]
    y_pred = np.squeeze(y_pred, axis=-1)
    y_pred = y_pred >= 0.5
    y_pred = y_pred.astype(np.int32)
    
    """ Saving the prediction """
    save_image_path = os.path.join("results2", img_name)
    save_results(image, y_pred, save_image_path)
    

2823it [25:45,  1.83it/s]


In [23]:
%cd /content/results
!zip -r results.zip .
from google.colab import files
files.download('results.zip')

/content/results
updating: 2641.jpg (deflated 8%)
updating: 750.jpg (deflated 6%)
updating: 897.jpg (deflated 8%)
updating: 2599.jpg (deflated 9%)
updating: 1191.jpg (deflated 6%)
updating: 2708.jpg (deflated 8%)
updating: 1300.jpg (deflated 7%)
updating: 2578.jpg (deflated 9%)
updating: 2656.jpg (deflated 8%)
updating: 804.jpg (deflated 7%)
updating: 2406.jpg (deflated 9%)
updating: 825.jpg (deflated 7%)
updating: 2035.jpg (deflated 9%)
updating: 2072.jpg (deflated 8%)
updating: 730.jpg (deflated 6%)
updating: 1332.jpg (deflated 6%)
updating: 2729.jpg (deflated 9%)
updating: 1261.jpg (deflated 6%)
updating: 2500.jpg (deflated 10%)
updating: 2210.jpg (deflated 8%)
updating: 2813.jpg (deflated 8%)
updating: 2430.jpg (deflated 8%)
updating: 930.jpg (deflated 9%)
updating: 2026.jpg (deflated 9%)
updating: 349.jpg (deflated 6%)
updating: 1661.jpg (deflated 9%)
updating: 140.jpg (deflated 8%)
updating: 547.jpg (deflated 6%)
updating: 1714.jpg (deflated 9%)
updating: 2683.jpg (deflated 8%)
u

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
%cd /content/files
!zip -r files.zip .
files.download('files.zip')

/content/files
updating: log.csv (deflated 52%)
updating: model.h5

In [34]:
%cd /content/results2
!zip -r results2.zip .
files.download('results2.zip')

/content/results2
  adding: 2641.jpg (deflated 4%)
  adding: 724.jpg (deflated 3%)
  adding: 1004.jpg (deflated 4%)
  adding: 1915.jpg (deflated 4%)
  adding: 750.jpg (deflated 3%)
  adding: 502.jpg (deflated 4%)
  adding: 2438.jpg (deflated 4%)
  adding: 729.jpg (deflated 3%)
  adding: 897.jpg (deflated 4%)
  adding: 1946.jpg (deflated 5%)
  adding: 2599.jpg (deflated 5%)
  adding: 1191.jpg (deflated 3%)
  adding: 1618.jpg (deflated 4%)
  adding: 911.jpg (deflated 4%)
  adding: 2708.jpg (deflated 5%)
  adding: 1306.jpg (deflated 4%)
  adding: 647.jpg (deflated 4%)
  adding: 1709.jpg (deflated 5%)
  adding: 580.jpg (deflated 3%)
  adding: 2318.jpg (deflated 4%)
  adding: 1165.jpg (deflated 4%)
  adding: 363.jpg (deflated 3%)
  adding: 606.jpg (deflated 3%)
  adding: 1300.jpg (deflated 4%)
  adding: 2578.jpg (deflated 5%)
  adding: 156.jpg (deflated 3%)
  adding: 1974.jpg (deflated 3%)
  adding: 1608.jpg (deflated 5%)
  adding: 299.jpg (deflated 4%)
  adding: 1606.jpg (deflated 5%)
  ad

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>